<a href="https://colab.research.google.com/github/MethmiDharmakeerthi/OurAcademicResearchIsBest/blob/main/Methmi/Methmi's_Modification_of_Gaya's_Server_Side_Cod_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python
!pip install tensorflow

In [ ]:
!pip install opencv-python tensorflow ffmpeg-python

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import ffmpeg



In [ ]:
# Input video file - slow motion penguin walking video

input_video = "/content/drive/MyDrive/Algorithm_Testing/slow-motion-penguin-walking-on-snow.mp4"

# Get video information using ffprobe
probe = ffmpeg.probe(input_video, v='error', select_streams='v:0',
                     show_entries='stream=width,height,bit_rate,r_frame_rate')
width = probe['streams'][0]['width']
height = probe['streams'][0]['height']
bit_rate = int(probe['streams'][0]['bit_rate'])
frame_rate = eval(probe['streams'][0]['r_frame_rate'])  # converts "24/1" to 24.0

print(f"Original Width: {width}")
print(f"Original Height: {height}")
print(f"Original Bitrate: {bit_rate} bits per second")
print(f"Original Frame Rate: {frame_rate}")

Original Width: 3840
Original Height: 2160
Original Bitrate: 37417661 bits per second
Original Frame Rate: 23.976023976023978


**Adjusting the parameters for compress the video**

In [ ]:
import ffmpeg

input_video = "/content/drive/MyDrive/Algorithm_Testing/slow-motion-penguin-walking-on-snow.mp4"
compressed_video = "/content/drive/MyDrive/Algorithm_Testing/compressed.mp4"

ffmpeg.input(input_video).output(
    compressed_video,
    r=15,                      # Reduce frame rate to 15 FPS
    video_bitrate='500k',      # Lower bitrate (e.g., 500 kbps)
    vf='format=yuv420p',       # Downscale color depth
    vcodec='libx264',          # Use H.264 codec for better compression
    preset='slow',             # Compression efficiency (options: ultrafast, fast, medium, slow, slower)
    crf=28                     # Constant Rate Factor (lower = better quality, 23 is default, 28 is lower quality but smaller size)
).run(overwrite_output=True)

(None, None)

**Compressed video info**

In [ ]:
# Input video file
input_video = "/content/drive/MyDrive/Algorithm_Testing/compressed.mp4"

# Get video information using ffprobe
probe = ffmpeg.probe(input_video, v='error', select_streams='v:0',
                     show_entries='stream=width,height,bit_rate,r_frame_rate')
width = probe['streams'][0]['width']
height = probe['streams'][0]['height']
bit_rate = int(probe['streams'][0]['bit_rate'])
frame_rate = eval(probe['streams'][0]['r_frame_rate'])  # converts "24/1" to 24.0

print(f"Original Width: {width}")
print(f"Original Height: {height}")
print(f"Original Bitrate: {bit_rate} bits per second")
print(f"Original Frame Rate: {frame_rate}")

Original Width: 3840
Original Height: 2160
Original Bitrate: 3056289 bits per second
Original Frame Rate: 15.0


In [ ]:
import os

output_dir = "/content/drive/MyDrive/Algorithm_Testing/frames"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
!ffmpeg -i "/content/drive/MyDrive/Algorithm_Testing/compressed.mp4" -qscale:v 2 "/content/drive/MyDrive/Algorithm_Testing/frames/frame_%04d.jpg"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

**Check the validation of the frames**

In [ ]:
import cv2

# Example: Open video and read frames
cap = cv2.VideoCapture(input_video)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab a frame. Exiting...")
        break

    if frame is not None:
        # Now apply bilateralFilter
        filtered = cv2.bilateralFilter(frame, d=9, sigmaColor=75, sigmaSpace=75)
        # Do something with the filtered frame...
    else:
        print("Empty frame. Skipping...")

Failed to grab a frame. Exiting...


**Enhance the Frames of the video**

In [ ]:
import cv2
import numpy as np
import os

input_frames_dir = "/content/drive/MyDrive/Algorithm_Testing/frames"
output_frames_dir = "/content/drive/MyDrive/Algorithm_Testing/enhanced_frames"

os.makedirs(output_frames_dir, exist_ok=True)

frame_files = sorted(os.listdir(input_frames_dir))

for frame_file in frame_files:
    frame_path = os.path.join(input_frames_dir, frame_file)
    frame = cv2.imread(frame_path)

    # --- Apply quality-preserving enhancement ---
    # 1. Bilateral Filter (for light denoising while keeping edges)
    filtered = cv2.bilateralFilter(frame, d=9, sigmaColor=75, sigmaSpace=75)

    # 2. Unsharp Mask (sharpen edges without overdoing it)
    gaussian = cv2.GaussianBlur(filtered, (9, 9), 10.0)
    unsharp = cv2.addWeighted(filtered, 1.5, gaussian, -0.5, 0)

    # 3. Mild contrast enhancement (optional)
    enhanced = cv2.convertScaleAbs(unsharp, alpha=1.1, beta=5)

    # Save result
    enhanced_frame_path = os.path.join(output_frames_dir, frame_file)
    cv2.imwrite(enhanced_frame_path, enhanced)

print(f"Enhanced frames are saved in {output_frames_dir}.")

Enhanced frames are saved in /content/drive/MyDrive/Algorithm_Testing/enhanced_frames.


**Convert to the video**

In [ ]:
import cv2
import os

# Input and output paths
input_frames_dir = "/content/drive/MyDrive/Algorithm_Testing/enhanced_frames"
output_video_path = "/content/drive/MyDrive/Algorithm_Testing/enhanced_video.mp4"

# Sorted list of frame files
frame_files = sorted(os.listdir(input_frames_dir))

# Load the first frame to get frame size
first_frame = cv2.imread(os.path.join(input_frames_dir, frame_files[0]))
height, width, _ = first_frame.shape

# Define video writer (use MP4V codec)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = 30  # Change this if your original video had a different FPS
video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Write all frames to the video
for frame_file in frame_files:
    frame_path = os.path.join(input_frames_dir, frame_file)
    frame = cv2.imread(frame_path)
    video_writer.write(frame)

video_writer.release()
print(f"Reconstructed video saved as: {output_video_path}")

Reconstructed video saved as: /content/drive/MyDrive/Algorithm_Testing/enhanced_video.mp4


**Enhanced Video Info**

In [ ]:
import ffmpeg

# Input video file
input_video = "/content/drive/MyDrive/Algorithm_Testing/enhanced_video.mp4"

# Get video information using ffprobe
probe = ffmpeg.probe(input_video, v='error', select_streams='v:0',
                     show_entries='stream=width,height,bit_rate,r_frame_rate')
width = probe['streams'][0]['width']
height = probe['streams'][0]['height']
bit_rate = int(probe['streams'][0]['bit_rate'])
frame_rate = eval(probe['streams'][0]['r_frame_rate'])  # converts "24/1" to 24.0

print(f"Original Width: {width}")
print(f"Original Height: {height}")
print(f"Original Bitrate: {bit_rate} bits per second")
print(f"Original Frame Rate: {frame_rate}")